# using Microsoft Graph to access Azure resources

This 2023 approach to authenticating and authorizing access to Azure Resources features the use of Microsoft Graph. “[Working with Azure Active Directory resources in Microsoft Graph](https://learn.microsoft.com/en-us/graph/api/resources/azure-ad-overview?view=graph-rest-1.0)” introduces this new _and familiar_ way of securing Azure.


What follows here focuses on the `DeviceCodeCredentialOptions` [📖 [docs](https://learn.microsoft.com/en-us/graph/sdks/create-client?tabs=CS)] technique that requires an Azure Active Directory user to manually log into Azure.

## NuGet packages

In [1]:
#r "nuget: Microsoft.Graph"
#r "nuget: Azure.Identity"

Installed Packages Azure.Identity, 1.9.0 Microsoft.Graph, 5.12.0

In [2]:
#r "nuget: SonghayCore"

Installed Packages SonghayCore, 6.0.4

## load conventional `ProgramMetadata`

In [3]:
using System.IO;
using System.Text.Json;
using Songhay.Models;

var json = File.ReadAllText(Environment.GetEnvironmentVariable("SONGHAY_APP_SETTINGS_PATH"));

JsonDocument.Parse(json).RootElement.TryGetProperty(typeof(ProgramMetadata).Name, out var element)

True

In [4]:
var meta = JsonSerializer.Deserialize<ProgramMetadata>(element);
var restMeta = meta.RestApiMetadataSet["RbacVault2023"];

restMeta.ClaimsSet.Keys

[ app-client-id, app-tenant-id, resource-scope, test-secret-id ]

## use `DeviceCodeCredential` for Azure Key Vault

In [5]:
using Azure.Core;
using Azure.Identity;

var options = new DeviceCodeCredentialOptions
{
    ClientId = restMeta.ClaimsSet["app-client-id"],
    TenantId = restMeta.ClaimsSet["app-tenant-id"],
    DeviceCodeCallback = (info, _) =>
        {
            // Display the device code message to
            // the user. This tells them
            // where to go to sign in and provides the
            // code to use.
            Console.WriteLine(info.Message);
            return Task.FromResult(0);
        }
};

var credential = new DeviceCodeCredential(options);

var scopes = new[] { restMeta.ClaimsSet["resource-scope"] };

var context = new TokenRequestContext(scopes);

var response = await credential.GetTokenAsync(context);

!string.IsNullOrWhiteSpace(response.Token)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RFH7PLX5K to authenticate.


True

In [6]:
using System.Net.Http;
using System.Net.Http.Headers;
using Songhay.Extensions;

var client = new HttpClient();
var uri = restMeta.ToUri("UriPathTemplateForTestSecret", restMeta.ClaimsSet["test-secret-id"]);

client.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", response.Token);

var secretResult = await client.GetAsync(uri);

var json = await secretResult.Content.ReadAsStringAsync();

var jDoc = JsonDocument.Parse(json);

jDoc.RootElement.GetProperty("value").GetString()

this-is-the-secret

## <!---->

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼